In [1]:
%cd <your folder path which contains data folder>

[WinError 123] The filename, directory name, or volume label syntax is incorrect: '<your folder path which contains data folder>'
G:\2021-1\Xử lý ảnh\Src\DemoOpenCV\Project\bai-3


In [2]:
import os

import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [3]:
image_folder = "./dtd/images"

label_file = "./dtd/labels/labels_joint_anno.txt"


In [4]:
def get_data(label_file: str):
    """
    Mapping image path with image class
    """
    mapped_data = []
    with open(label_file, 'r') as f:
        data = f.readlines()
        for line in data:
            line_split = line.split(' ')
            image_path = os.path.join(image_folder, line_split[0])
            image_class = line_split[1]
            mapped_data.append((image_path, image_class))
    return mapped_data


In [5]:
# alexnet model
# reference: https://pytorch.org/vision/main/_modules/torchvision/models/alexnet.html
alexnet = models.alexnet(pretrained=True)

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(170),
    transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]
)
img_to_tensor = transforms.ToTensor()

In [6]:
train_image_paths = []
y_train = []
test_image_paths = []
y_test = []

data = get_data(label_file)

train_size = int(len(data) * 0.7)
print("Number train images: ", train_size)
print("Number test images: ", len(data) - train_size)
# random dataset and seperate them to train dataset and test dataset
np.random.shuffle(data)
train_data = data[: train_size]
test_data = data[train_size:]

for el in train_data:
    train_image_paths.append(el[0])
    y_train.append(el[1])

for el in test_data:
    test_image_paths.append(el[0])
    y_test.append(el[1])

Number train images:  3947
Number test images:  1693


In [7]:
class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x


print(alexnet.classifier)
# Changing the output of the Alexnet model to output a 4096 dimensional feature vector
alexnet.classifier[6] = Identity()

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [8]:
all_train_image_features = torch.zeros((len(y_train), 4096))
for i in range(train_size):
    # load image
    img = Image.open(train_image_paths[i])
    # resize image to 150x150, if you have many resource of hardware, you can increate size of image to 300x300 => increase accuracy
    img = img.resize((150, 150))
    img_tensor = img_to_tensor(img).unsqueeze(0)
    features = alexnet(img_tensor)
    all_train_image_features[i, :] = alexnet(img_tensor)
# Save train data to Train_data.npz
np.savez('Train_data.npz', x_train_all_features=all_train_image_features.detach().numpy(), y_train=y_train)


In [ ]:
all_test_image_features = torch.zeros((len(y_test), 4096))
for i in range(len(test_data)):
    img = Image.open(test_image_paths[i])
    img = img.resize((150, 150))
    img_tensor = img_to_tensor(img).unsqueeze(0)
    features = alexnet(img_tensor)
    all_test_image_features[i, :] = alexnet(img_tensor)

# Save test data to Test_data.npz
np.savez('Test_data.npz', x_test_all_features=all_test_image_features.detach().numpy(), y_test=y_test)
